In [20]:
import pandas as pd
import re


In [9]:
df = pd.read_csv('output.txt')

In [11]:
df.columns

Index(['unique_number', 'name', 'reg_no'], dtype='object')

In [16]:
df['unique_number'] = df['unique_number'].str.replace(r'\.jpg$', '', case=False, regex=True)

In [17]:
df

,unique_number,name,reg_no
0,ICMAI11010027,Mehak Sahu Cma R,1232047736
1,ICMAI11178567,Himaniben Ashwinkumar Patel Cma,1241060964
2,ICMAI11245933,Harsh Sunil Makhujja Cma R,1232046446
3,ICMAI11705988,Not Found,3241070193
4,ICMAI11776670,Adithyan M K Cma R,2232054596
...,...,...,...
285,ICMAI98544137,P Rahul Cma R,2232040743
286,ICMAI98862767,Shivani Kumari Cma R,4231029086
287,ICMAI99147408,Yash Kushwaha Cma R,4241057319
288,ICMAI99408635,Hafsa Zaheer Wangde Cma R,1241060233


In [36]:
#to clean the name , that comes with the Cma , Cma R , Cma reg , things. 

df['name'] = df['name'].str.replace(r'\bCMA\s*R.*$', '', flags=re.IGNORECASE, regex=True)
df['name'] = df['name'].str.replace(r'\bCMA\s*.*$', '', flags=re.IGNORECASE, regex=True)
df['name'] = df['name'].str.replace(r'\bCMA\s*REG.*$', '', flags=re.IGNORECASE, regex=True)
df['name'] = df['name'].str.strip()


In [ ]:
#to save the updated csv file . Preprocessed , with removal of noice . 
df.to_csv("output.txt", index=False)

In [25]:
df2 = pd.read_csv('output.txt')

In [33]:
df2[df2.name == 'Not Found'].name

3      Not Found
147    Not Found
176    Not Found
202    Not Found
209    Not Found
Name: name, dtype: object

In [35]:
#in last to make it , only clean data , we got from the script  . 

import pandas as pd

# Load the data
df = pd.read_csv("output.txt")

# Detect the name column (like 'Name', 'Full Name', etc.)
name_col = [col for col in df.columns if 'name' in col.lower()]
if not name_col:
    raise Exception(" Couldn't find a column with 'name' in header.")
name_col = name_col[0]

# 1. Extract rows where name == 'Not Found'
df_not_found = df[df[name_col] == 'Not Found']
df_not_found.to_csv("output2.txt", index=False)
print(f"Saved {len(df_not_found)} rows to output2.txt")

# 2. Remove those rows from original data
df_cleaned = df[df[name_col] != 'Not Found']
df_cleaned.to_csv("output.txt", index=False)
print(f"Removed them from output.txt, remaining: {len(df_cleaned)} rows")


Saved 0 rows to output2.txt
Removed them from output.txt, remaining: 285 rows


In [ ]:

#to check the output and all . 

import pandas as pd

# Load the CSV files
file1 = pd.read_csv("output.txt", header=None)   #output.txt
file2 = pd.read_csv("icmai_certificate_data.csv", header=None)

# Print the shapes for debug
print("File1 shape:", file1.shape)
print("File2 shape:", file2.shape)

# Check if the files have the expected number of columns
if file1.shape[1] < 3 or file2.shape[1] < 9:
    raise ValueError("One of the files does not have the required number of columns.")

# Strip whitespaces from relevant columns in both files
file1[0] = file1[0].astype(str).str.strip()  # unique_number
file1[1] = file1[1].astype(str).str.strip()  # name
file1[2] = file1[2].astype(str).str.strip()  # reg_no

file2[0] = file2[0].astype(str).str.strip()  # name
file2[4] = file2[4].astype(str).str.strip()  # unique_number
file2[5] = file2[5].astype(str).str.strip()  # reg_no

# List to store rows with mismatches
error_rows = []

# Iterate through each row in file2
for index, row in file2.iterrows():
    # Match file2[unique_number] (column 5) with file1[unique_number] (column 1)
    match_rows = file1[file1[0] == row[4]]
    
    if not match_rows.empty:
        # For each match found, check name and reg_no
        for _, match_row in match_rows.iterrows():
            condition1 = match_row[1].strip() == row[0].strip()  # file1[name] == file2[name]
            condition2 = match_row[2].strip() == row[5].strip()  # file1[reg_no] == file2[reg_no]
            
            # If either condition fails, add the row to error_rows
            if not (condition1 and condition2):
                error_rows.append(row.tolist())
    else:
        # If no match found for unique_number, add the row to error_rows
        error_rows.append(row.tolist())

# Write mismatched rows to error.txt
with open("error.txt", "w") as f:
    for err in error_rows:
        f.write(",".join(map(str, err)) + "\n")

print("Done. Mismatched rows written to error.txt.")


File1 shape: (286, 3)
File2 shape: (172, 9)
Done. Mismatched rows written to error.txt.
